In [1]:
from dangin import Stitcher, dataloader_dmap, dataloader_ply
import numpy as np
import copy
import open3d as o3d
import pathlib

In [2]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is for stitching NEW point cloud with PREVIOUS. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***

root_path = str(pathlib.Path('').parent.resolve().parent)

# Select one dataset
dataset = 'LA200'
dataset_path = root_path + f'/datasets/{dataset}'

# Represents how many individual point clouds are going to be grouped
rolling_window = 5
pcd_size = 128*128
rolling_window_total_points = rolling_window * pcd_size
    
stitcher = Stitcher()
dataset = dataloader_dmap(dataset_path, ground_truth=True)
point_cloud_0 = o3d.geometry.PointCloud()
point_cloud_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)
stitcher.generate_pointcloud(img_0, dmap_0, point_cloud_0)
point_cloud_0 = stitcher.outlier_removal(point_cloud_0)

history_point_cloud = history_point_cloud_down = copy.deepcopy(point_cloud_0)
history_points_xyz = np.array(history_point_cloud.points) 

vis = o3d.visualization.Visualizer()
vis.create_window(height=1000, width=1000)
vis.add_geometry(history_point_cloud_down)

print("Image: 0")
for i, package in enumerate(dataset):
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, point_cloud_1)
    point_cloud_1 = stitcher.outlier_removal(point_cloud_1)
    
    point_cloud_1_temp = copy.deepcopy(point_cloud_1)
    
    if len(history_points_xyz) >= (rolling_window_total_points):
        pass
    else:
        stitcher.point_cloud_init_pose(point_cloud_0, stitcher.current_transform)
        
    stitcher.point_cloud_init_pose(point_cloud_1, stitcher.current_transform)                     # As the name implies first initialize both pcls on specified loc.
    result_icp = stitcher.global_and_icp_registration_v2(point_cloud_1, point_cloud_0)                 # Calculate the best transform starting between both point clouds
    current_transform = stitcher.current_transform @ result_icp.transformation                         # Calculate the new current transform of the trayectory. 
    stitcher.set_current_transform(current_transform)                                                  # This is useful for the next CONSECUTIVE pcl. As the starting loc.
    
    history_point_cloud += point_cloud_1.transform(result_icp.transformation)
    
    vis.update_geometry(history_point_cloud_down)
    vis.poll_events()
    vis.update_renderer()
    
    history_points_xyz = np.array(history_point_cloud.points)
    
    if len(history_points_xyz) >= (rolling_window_total_points): 
        point_cloud_0 = o3d.geometry.PointCloud()
        point_cloud_0.points = o3d.utility.Vector3dVector(history_points_xyz[-(rolling_window_total_points):, :])
    else:
        point_cloud_0 = copy.deepcopy(point_cloud_1_temp)

    print(f"Image: {i+1}")
    if (i+1) == 199:
        break

vis.destroy_window()
o3d.visualization.draw_geometries([history_point_cloud])

Image: 0
Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10
Image: 11
Image: 12
Image: 13
Image: 14
Image: 15
Image: 16
Image: 17
Image: 18
Image: 19
Image: 20
Image: 21
Image: 22
Image: 23
Image: 24
Image: 25
Image: 26
Image: 27
Image: 28
Image: 29
Image: 30
Image: 31
Image: 32
Image: 33
Image: 34
Image: 35
Image: 36
Image: 37
Image: 38
Image: 39
Image: 40
Image: 41
Image: 42
Image: 43
Image: 44
Image: 45
Image: 46
Image: 47
Image: 48
Image: 49
Image: 50
Image: 51
Image: 52
Image: 53
Image: 54
Image: 55
Image: 56
Image: 57
Image: 58
Image: 59
Image: 60
Image: 61
Image: 62
Image: 63
Image: 64
Image: 65
Image: 66
Image: 67
Image: 68
Image: 69
Image: 70
Image: 71
Image: 72
Image: 73
Image: 74
Image: 75
Image: 76
Image: 77
Image: 78
Image: 79
Image: 80
Image: 81
Image: 82
Image: 83
Image: 84
Image: 85
Image: 86
Image: 87
Image: 88
Image: 89
Image: 90
Image: 91
Image: 92
Image: 93
Image: 94
Image: 95
Image: 96
Image: 97
Image: 98
Image: 99
Image: 100